# VRR Trend Constant Vol vs Varying Vol
This notebok runs analysis on the VRR Trend strategy.

## Import Libraries

In [ ]:
import os
os.chdir('..')

import plotly.express as px
import plotly.graph_objects as go
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
import pandas as pd

## Import Equity & FI Benchmark Data

In [ ]:
equity_bmk = 'SPTR'
include_fi = True
strat_drop_list = ['99%/90% Put Spread','Down Var', 'Vortex','VOLA', 'Dynamic Put Spread', 'VRR',
       'GW Dispersion', 'Corr Hedge', 'Def Var']

#create returns data dictionary for equity benchmark
ret_dict= dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Import VRR Trend iterations and Display Historical Performance

In [ ]:
vrr_trend = pd.read_excel(dm.NEW_DATA+'VRR Trend with Constant Vol.xlsx', sheet_name='data', index_col=0)
vrr_trend_dict = dm.merge_dicts(ret_dict, dm.get_data_dict(vrr_trend))
fig = go.Figure()
for strat in vrr_trend.columns:
    fig.add_trace(go.Scatter(x = pd.to_datetime(vrr_trend.index, format = '%m/%d/%Y'),
                             y = vrr_trend[strat],mode = 'lines',name = strat))
fig.update_layout(
    title={
        'text': "<b>Historical Performance</b>",
        'y':0.9,
        'x':0.5,'xanchor': 'center','yanchor': 'top'
    },
    title_font_family="Calibri",
    titlefont = {"size":20},
    xaxis = dict(tickfont = dict(size=14)),
    showlegend=True,
    plot_bgcolor='White'
    )
fig.update_xaxes(showline=True,linewidth=2,linecolor='black',mirror=False)
fig.update_yaxes(showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()

# Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict_trend = summary.get_corr_data(vrr_trend_dict, include_fi=include_fi)

# Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **corr** - Show:
    * **full** - full history correaltions
    * **equity_down** - risk seeking benchmark downside returns correlations or 
    * **equity_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict_trend[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict_trend = summary.get_analytics_data(vrr_trend_dict, include_fi=True)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_trend[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict_trend = summary.get_hist_data(vrr_trend_dict)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict_trend[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **bmk** - Equity or FI bmk

In [ ]:
group_list=['Quintile','Decile']
bmk_list = list(vrr_trend_dict['Monthly'].select_dtypes('float').columns)[0:2]
@interact
def display_returns(group = group_list, bmk = bmk_list):
    quintile_df = summary.get_grouped_data(vrr_trend_dict, group = group, strat = bmk)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list,bmk = bmk_list, strat=list(vrr_trend_dict['Monthly'].select_dtypes('float').columns)[2:]):    
    df = summary.get_grouped_data(vrr_trend_dict, group = group, strat=bmk)
    
    for col in df.columns:
        if not (col == bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    

## VRR+Trend

## Add VRR+Trend tracks and merge to Bmk

In [ ]:
vrr_w_trend = vrr_trend.copy()
vrr_weight = .75
vrr_trend_weight = .25

vrr_w_trend['VRR+Trend-50% const'] = vrr_w_trend['VRR']*vrr_weight + vrr_w_trend['Constant vol at entry (ER)']*vrr_trend_weight
vrr_w_trend['VRR+Trend-50% vary'] = vrr_w_trend['VRR']*vrr_weight + vrr_w_trend['VRR Trend (ER)']*vrr_trend_weight
vrr_w_trend['VRR+Trend-25% const'] = vrr_w_trend['VRR']*vrr_weight + vrr_w_trend['25% vol - constant (ER)']*vrr_trend_weight
vrr_w_trend['VRR+Trend-25% vary'] = vrr_w_trend['VRR']*vrr_weight + vrr_w_trend['25% vol - varying (ER)']*vrr_trend_weight
vrr_w_trend['VRR+Trend-75% const'] = vrr_w_trend['VRR']*vrr_weight + vrr_w_trend['75% vol - constant (ER)']*vrr_trend_weight
vrr_w_trend['VRR+Trend-75% vary'] = vrr_w_trend['VRR']*vrr_weight + vrr_w_trend['75% vol - varying (ER)']*vrr_trend_weight
vrr_w_trend = vrr_w_trend[['VRR+Trend-50% const','VRR+Trend-50% vary','VRR+Trend-25% const','VRR+Trend-25% vary',
                           'VRR+Trend-75% const','VRR+Trend-75% vary']]
vrr_w_trend_dict = dm.get_data_dict(vrr_w_trend)
vrr_w_trend_dict = dm.merge_dicts(ret_dict, vrr_w_trend_dict)

# Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict = summary.get_corr_data(vrr_w_trend_dict, include_fi=include_fi)

# Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **corr** - Show:
    * **full** - full history correaltions
    * **equity_down** - risk seeking benchmark downside returns correlations or 
    * **equity_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(vrr_w_trend_dict,include_fi=True)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(vrr_w_trend_dict)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **bmk** - Equity or FI bmk

In [ ]:
group_list=['Quintile','Decile']
bmk_list = list(vrr_w_trend_dict['Monthly'].select_dtypes('float').columns)[0:2]
@interact
def display_returns(group = group_list, bmk = bmk_list):
    quintile_df = summary.get_grouped_data(vrr_w_trend_dict, group = group, strat = bmk)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list,bmk = bmk_list, strat=list(vrr_w_trend_dict['Monthly'].select_dtypes('float').columns)[2:]):    
    df = summary.get_grouped_data(vrr_w_trend_dict, group = group, strat=bmk)
    
    for col in df.columns:
        if not (col == bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    